In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv("IMDB_Dataset.csv")

In [3]:
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/riyagupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data = []

In [6]:
for i in range(0, 50000):
    info = dataset["review"][i]
    info = re.sub('[^a-zA-Z]', ' ', info)
    info = info.lower()
    info = info.split()
    
    info = [ps.stem(word) for word in info if not word in set(stopwords.words('english'))]
    info = ' '.join(info)
    data.append(info)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_features=2500)
x = cv.fit_transform(data).toarray()
y = dataset.iloc[:,1:2].values

In [11]:
x.shape

(51347, 2500)

In [12]:
y.shape

(50000, 1)

In [14]:
x = x[0:50000]

In [15]:
x.shape

(50000, 2500)

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [18]:
print(y_test)

[1 0 0 ... 1 1 0]


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [20]:
model = Sequential()
model.add(Dense(units = 2500, kernel_initializer = "random_uniform", activation = "relu"))
model.add(Dense(units = 5000, kernel_initializer = "random_uniform", activation = "relu"))
model.add(Dense(units = 5000, kernel_initializer = "random_uniform", activation = "relu"))
model.add(Dense(units = 1, kernel_initializer = "random_uniform", activation = "sigmoid"))
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = ['accuracy'])
model.fit(x_train, y_train, epochs = 5)

Epoch 1/5
1250/1250 [==============================] - 358s 285ms/step - loss: 0.8176 - accuracy: 0.5013
Epoch 2/5
1250/1250 [==============================] - 398s 318ms/step - loss: 0.6921 - accuracy: 0.5403
Epoch 3/5
1250/1250 [==============================] - 447s 358ms/step - loss: 0.5616 - accuracy: 0.7259
Epoch 4/5
1250/1250 [==============================] - 470s 376ms/step - loss: 0.2876 - accuracy: 0.9028
Epoch 5/5
1250/1250 [==============================] - 488s 390ms/step - loss: 0.1954 - accuracy: 0.9543


In [21]:
pred = model.predict(x_test)
pred = pred>0.5

In [22]:
pred

array([[False],
       [ True],
       [False],
       ...,
       [ True],
       [False],
       [False]])

In [23]:
yp = model.predict(cv.transform(["bad"]))

In [24]:
yp

array([[0.5008688]], dtype=float32)

In [25]:
yp>0.5

array([[ True]])

In [26]:
#row 8 - positive
text = "I sure would like to see a resurrection of a up dated Seahunt series with the tech they have today it would bring back the kid excitement in me.I grew up on black and white TV and Seahunt with Gunsmoke were my hero's every week.You have my vote for a comeback of a new sea hunt.We need a change of pace in TV and this would work for a world of under water adventure.Oh by the way thank you for an outlet like this to view many viewpoints about TV and the many movies.So any ole way I believe I've got what I wanna say.Would be nice to read some more plus points about sea hunt.If my rhymes would be 10 lines would you let me submit,or leave me out to be in doubt and have me to quit,If this is so then I must go so lets do it."

text = re.sub('[^a-zA-Z]', ' ', text)
text = text.lower()
text = text.split()

text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
text = ' '.join(text)

In [27]:
yp = model.predict(cv.transform([text]))
yp>0.5

array([[ True]])